# PID

In [1]:
import logging
logging.basicConfig(format='%(levelname)s: %(message)s', level=logging.INFO)  # set level to INFO for wordy
import matplotlib.pyplot as plt
from IPython.display import HTML

import numpy as np
import jax.numpy as jnp

from extravaganza.dynamical_systems import PIDGym

from extravaganza.lifters import NoLift, RandomLift, LearnedLift
from extravaganza.sysid import SysID
from extravaganza.controllers import LiftedBPC, ConstantController
from extravaganza.rescalers import ADAM, D_ADAM, DoWG
from extravaganza.utils import ylim, render
from extravaganza.experiments import Experiment

# seeds for randomness. setting to `None` uses random seeds
SYSTEM_SEED = None
CONTROLLER_SEED = None
LIFTER_AND_SYSID_SEED = None

## System

describe here

## Hyperparameters

In [4]:
name = 'pid_both'
filename = '../logs/{}.pkl'.format(name)

def get_experiment_args():
    # --------------------------------------------------------------------------------------
    # ------------------------    EXPERIMENT HYPERPARAMETERS    ----------------------------
    # --------------------------------------------------------------------------------------

    num_trials = 1
    T = 20000
    T0 = 10000
    reset_condition = lambda t: t == T0
    use_multiprocessing = False
    render_every = None

    # --------------------------------------------------------------------------------------
    # --------------------------    SYSTEM HYPERPARAMETERS    ------------------------------
    # --------------------------------------------------------------------------------------

    du = 2  # control dim
    initial_Kp = 0.1  # initial value for Kp
    initial_Kd = 0.  # initial value for Kp
    def apply_control(control, system): 
        system.pid.Kp = system.pid.Kp.at[0].set(control[0].item())     
        system.pid.Kd = system.pid.Kd.at[0].set(control[1].item())        
        
    env_name = 'CartPole-v1'  
    # env_name = 'MountainCarContinuous-v0'

    system_args = {
        'env_name': env_name,
        'apply_control': apply_control,
        'control_dim': du,  # because CartPole
        'repeat': 10,
        'gym_repeat': 1,
        'max_episode_len': 300,
        'seed': SYSTEM_SEED,
        'Kp': initial_Kp,
        'Ki': 0.01,
        'Kd': initial_Kd
    }
    make_system = lambda : PIDGym(**system_args)

    # --------------------------------------------------------------------------------------
    # ------------------------    LIFT/SYSID HYPERPARAMETERS    ----------------------------
    # --------------------------------------------------------------------------------------

    sysid_method = 'regression'
    sysid_scale = 1.

    learned_lift_args = {
        'lift_lr': 0.01,
        'sysid_lr': 0.01,
        'cost_lr': 0.001,
        'depth': 6,
        'buffer_maxlen': int(1e6),
        'num_epochs': 100,
        'batch_size': 64,
        'seed': LIFTER_AND_SYSID_SEED
    }

    # --------------------------------------------------------------------------------------
    # ------------------------    CONTROLLER HYPERPARAMETERS    ----------------------------
    # --------------------------------------------------------------------------------------

    h = 20  # controller memory length (# of w's to use on inference)
    hh = 20  # history length of the cost/control histories
    lift_dim = 40  # dimension to lift to

    m_update_rescaler = lambda : ADAM(0.003, betas=(0.9, 0.99))
    m0_update_rescaler = lambda : ADAM(0.003, betas=(0.9, 0.99))
    k_update_rescaler = lambda : ADAM(0.003, betas=(0.9, 0.99))

    lifted_bpc_args = {
        'h': h,
        'method': 'REINFORCE',
        'initial_scales': (0.01, 0.1, 0.01),  # M, M0, K   (uses M0's scale for REINFORCE)
        'rescalers': (m_update_rescaler, m0_update_rescaler, k_update_rescaler),
        'T0': T0,
        'bounds': ([0, 0], [1, 1]),
        'initial_u': jnp.array([initial_Kp, initial_Kd]),
        'decay_scales': False,
        'use_tanh': False,
        'use_K_from_sysid': False,
        'seed': CONTROLLER_SEED
    }

    make_controllers = {
#         '0.0': lambda sys: ConstantController(0.0, du),
#         '0.1': lambda sys: ConstantController(0.1, du),
#         '0.4': lambda sys: ConstantController(0.4, du),
#         '0.7': lambda sys: ConstantController(0.7, du),
#         '1.0': lambda sys: ConstantController(1.0, du),

#         'No Lift': lambda sys: LiftedBPC(lifter=NoLift(hh, du, LIFTER_AND_SYSID_SEED), sysid=SysID(sysid_method, du, hh, sysid_scale, LIFTER_AND_SYSID_SEED), **lifted_bpc_args),
    #     'Random Lift': lambda sys: LiftedBPC(lifter=RandomLift(hh, du, lift_dim, learned_lift_args['depth'], LIFTER_AND_SYSID_SEED), sysid=SysID(sysid_method, du, lift_dim, sysid_scale, LIFTER_AND_SYSID_SEED), **lifted_bpc_args),
        'Learned Lift': lambda sys: LiftedBPC(lifter=LearnedLift(hh, du, lift_dim, scale=sysid_scale, **learned_lift_args), **lifted_bpc_args)
    }
    experiment_args = {
        'make_system': make_system,
        'make_controllers': make_controllers,
        'num_trials': num_trials,
        'T': T,
        'reset_condition': reset_condition,
        'reset_seed': SYSTEM_SEED,
        'use_multiprocessing': use_multiprocessing,
        'render_every': render_every,
    }
    return experiment_args

## actually run the thing :)

In [5]:
# run
experiment = Experiment(name)
stats = experiment(get_experiment_args)

INFO: (EXPERIMENT) --------------------------------------------------
INFO: (EXPERIMENT) ----------------- TRIAL 0 -----------------------
INFO: (EXPERIMENT) --------------------------------------------------

INFO: (EXPERIMENT): testing Learned Lift
 50%|████████████████████████████▍                            | 9998/20000 [06:08<13:06, 12.71it/s, control=[0.4832611  0.20772418], cost=-22]INFO: (LIFTEDBPC) Note that we are only updating lifter during sysid phase
INFO: (LEARNEDLIFT): training!
INFO: (LEARNEDLIFT) 	mean loss for past 25 epochs was 0.01956543408334255
 50%|████████████████████████████▍                            | 9998/20000 [06:19<13:06, 12.71it/s, control=[0.4832611  0.20772418], cost=-22]INFO: (LEARNEDLIFT) 	mean loss for past 25 epochs was 0.0034595388825982808
INFO: (LEARNEDLIFT) 	mean loss for past 25 epochs was 0.0026573001500219107
INFO: (LEARNEDLIFT) 	mean loss for past 25 epochs was 0.002890933221206069
INFO: (LEARNEDLIFT) 	mean loss for past 25 epochs was 0.00

In [ ]:
# save args and stats!  --  note that to save the args, we actually save the `get_args` function. we can print the 
#                           source code later to see the hyperparameters we chose
experiment.save(filename)

## Visualization
We keep track of the useful information through `Stats` objects, which can `register()` a variable to keep track of (which it does via calls to `update()`) and which can be aggregated via `Stats.aggregate()` for mean and variance statistics. 

We define below a plotting arrangement that plots all the desired quantities from both the system and controller.

In [ ]:
def plot_pid(experiment: Experiment):
    assert experiment.stats is not None, 'cannot plot the results of an experiment that hasnt been run'
    all_stats = experiment.stats
    
    # clear plot and calc nrows
    plt.clf()
    n = 5
    nrows = n + (len(all_stats) + 1) // 2
    fig, ax = plt.subplots(nrows, 2, figsize=(16, 6 * nrows))

    # plot system stats
    for i, (method, stats) in enumerate(all_stats.items()):
        if stats is None: 
            print('WARNING: {} had no stats'.format(method))
            continue
        stats.plot(ax[0, 0], 'P', label=method)
#         stats.plot(ax[0, 1], 'ws', label=method)
        stats.plot(ax[1, 0], 'us', label=method)
        stats.plot(ax[1, 1], 'Kp', label=method + ' Kp')
        stats.plot(ax[1, 1], 'Kd', label=method + ' Kd')

        stats.plot(ax[2, 0], 'rewards', label=method)
        stats.plot(ax[2, 1], 'avg rewards since reset', label=method)

        stats.plot(ax[3, 0], '||A||_op', label=method)
        stats.plot(ax[3, 1], '||B||_F', label=method)
        stats.plot(ax[4, 0], '||A-BK||_op', label=method)
        stats.plot(ax[4, 1], 'cost diffs', label=method)
        i_ax = ax[n + i // 2, i % 2]
        stats.plot(ax[0, 1], 'disturbances', label=method)
        stats.plot(i_ax, 'K @ state', label='K @ state')
        stats.plot(i_ax, 'M \cdot w', label='M \cdot w')
        stats.plot(i_ax, 'M0', label='M0')
        i_ax.set_title('u decomp for {}'.format(method))
        i_ax.legend()

    # set titles and legends and limits and such
    # (note: `ylim()` is so useful! because sometimes one thing blows up and then autoscale messes up all plots)
    _ax = ax[0, 0]; _ax.set_title('error'); _ax.legend()
    _ax = ax[0, 1]; _ax.set_title('disturbances'); _ax.legend()
    _ax = ax[1, 0]; _ax.set_title('controls'); _ax.legend()
    _ax = ax[1, 1]; _ax.set_title('state'); _ax.legend()
    
    _ax = ax[2, 0]; _ax.set_title('instantaneous rewards'); _ax.legend()
    _ax = ax[2, 1]; _ax.set_title('avg rewards since last reset'); _ax.legend()
    
    _ax = ax[3, 0]; _ax.set_title('||A||_op'); _ax.legend()
    _ax = ax[3, 1]; _ax.set_title('||B||_F'); _ax.legend()
    
    _ax = ax[4, 0]; _ax.set_title('||A-BK||_op'); _ax.legend()
    _ax = ax[4, 1]; _ax.set_title('cost diffs'); _ax.legend()
    pass

### Plot

In [ ]:
plot_pid(experiment)
# # plot
# from copy import deepcopy
# e = deepcopy(experiment)
# e.stats = {k: s[4100:] for k, s in e.stats.items() if k not in ['No Lift', 'RBPC']}
# plot_pid(e)

### Dynamic Plot

In [ ]:
# dynamic plot
anim = render(experiment, 'Kp', 'rewards', sliderkey='us', save_path=None, duration=5)
vid = anim.to_html5_video()
HTML(vid)